In [ ]:
import warnings
warnings.filterwarnings('ignore') # hide warnings
import os
import pandas as pd
import geopandas as gpd
import numpy as np
import sys
sys.path.append("../..")
from dyntapy.supply_data import road_network_from_place, relabel_graph
from dyntapy.demand_data import add_centroids, od_graph_from_matrix
from dyntapy.assignments import StaticAssignment
from pyproj import Proj, transform
from dyntapy.toll import create_toll_object

buffer = 40
city = 'BRUSSEL'

HERE = os.path.dirname(os.path.realpath("__file__"))
data_path = HERE + os.path.sep + os.pardir + os.path.sep + 'data_map' + os.path.sep + 'STA'
zoning_path = data_path + os.path.sep + 'shapefiles' + os.path.sep + city + "_" + str(buffer) + "_10" + os.path.sep + city + "_" + str(buffer) + "_10.shp"
od_path = data_path + os.path.sep + 'OD_matrix' + os.path.sep + city + "_" + str(buffer) + "_9_.xlsx"

In [ ]:
network = road_network_from_place(city, buffer_dist_close=buffer*1000)
network = relabel_graph(network)

In [ ]:
zoning = gpd.read_file(zoning_path) 
od = pd.read_excel(od_path)
od_array = od.to_numpy() # The OD matrix is now stored in a numpy array

# Retrieve zone number, x_centroid (LON) and y_centroid (LAT) from each zone. 
zone_numbers = zoning["ZONENUMMER"]
x_lamb = zoning["X_LAMB"]
x_lamb = x_lamb.to_numpy()
y_lamb = zoning["Y_LAMB"]
y_lamb = y_lamb.to_numpy()

# Project to correct CRS. 
inProj = Proj(init='epsg:31370')
outProj = Proj(init='epsg:4326')
x_centroids, y_centroids = transform(inProj,outProj,x_lamb,y_lamb)

# Add the centroids to the network. Relabelling the graph is required (see demo for reason why)
connector_type = 'link'
network = add_centroids(network, x_centroids,y_centroids,k=1, method=connector_type)
network = relabel_graph(network)

# Create OD graph and plot demand
od_graph = od_graph_from_matrix(od_array,x_centroids,y_centroids) 
# demand plot is useless since there are too many OD-pairs 

In [ ]:
assignment = StaticAssignment(network,od_graph)
result = assignment.run('dial_b')
print(result.link_costs[5:13])

In [ ]:
toll_method = 'single'
toll_link_id = 9
toll_value = 10
toll_object = create_toll_object(network, toll_method, toll_link_id, toll_value)
assignment2 = StaticAssignment(network,od_graph, toll_object)
result2 = assignment2.run('dial_b')

In [ ]:
print(result2.link_costs[5:13])

In [ ]:
toll_method = 'cordon'
toll_link_id = [5, 9, 10]
toll_value = 10
toll_object2 = create_toll_object(network, toll_method, toll_link_id, toll_value)
assignment3 = StaticAssignment(network,od_graph, toll_object2)
result3 = assignment3.run('dial_b')
print(result3.link_costs[5:13])

In [ ]:
toll_method = 'cordon'
toll_link_id = [5, 9, 10]
toll_value = [10, 20, 30]
toll_object3 = create_toll_object(network, toll_method, toll_link_id, toll_value)
assignment4 = StaticAssignment(network,od_graph, toll_object3)
result4 = assignment4.run('dial_b')
print(result4.link_costs[5:13])

In [ ]:
toll_method = 'zone'
toll_link_id = [5, 9, 10,12]
toll_value = 3
toll_object4 = create_toll_object(network, toll_method, toll_link_id, toll_value)
assignment5 = StaticAssignment(network,od_graph, toll_object4)
result5 = assignment5.run('dial_b')
print(result5.link_costs[5:13])